<a href="https://colab.research.google.com/github/chengwen-zheng/openAi-lab/blob/main/llama_index_to_read_a_book.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# setup

In [ ]:
%pip install llama-index
%pip install langchain
%pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 kB 7.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 548.8/548.8 kB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.3/70.3 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 64.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 55.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 47.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.2/68.2 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.3/269.3 kB 2

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

# how to red file in Google Drive

In [ ]:
# Import PyDrive and associated libraries.
# This only needs to be done once per notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# List .txt files in the root.
#
# Search query reference:
# https://developers.google.com/drive/v2/web/search-parameters
listed = drive.ListFile().GetList()
for file in listed:
  print('title {}, id {}'.format(file['title'], file['id']))

title llama_index_to_read_a_book.ipynb, id 1DQDRA1pyRlDNeyTE8gr2Um4h5yWp8BrS
title test-openai.ipynb, id 1wHIqeZB675Kr8QnXEELCkmGdt_T_H3xj
title config.json, id 14x1k7ErttIPXRcKN53g_Ogn3zfn6AEM_
title mr_fujino.txt, id 1Z_X79_FrTW69h4SND9BnT9qr3ynsdl5G
title Untitled, id 1I_xhJt_7U59myQaxLAMtlUnz_xmDMSa8
title semantic_search.ipynb, id 1MXkSYu3eavOHrr4FwfQ-FoBVs3LE3bQB
title edit_moderation_text.ipynb, id 1PTfZGQOUWWMDQ41Xl9KGUxI270EMGunn
title text-clusters.ipynb, id 1bYXHDDARDf-pJiSWAttiArQGk_VxE1s0
title Fasttext-lab.ipynb, id 17YcjW4HEBGEToNu1ZAqMT-iOrKeMf-Ax
title T5.ipynb, id 1LLV0UCHCmXJ16diU5-jmkh47QysRB6SW
title Colab Notebooks, id 1hjY5nzNXXO7Q9xn449qgGaVZmnW8aCic
title React Server Components, id 1a4yskaYi7Ons0LphfyBbFd3t32c39rJDOxv0WwAALBM
title rust学习笔记, id 1XP8BLuZmnKMMfiuuN-lXqF8mm8J3R_cHz-pP-T0q36g
title ChatGpt相关资料, id 1ky1sQiAZcJPk9FduOzoMrrok7JzHox3SGCLKXn0da3k
title 提案, id 18CHXMKdoSeuP5q2eTS6u5ECPQcdhzF2wiDQxte7PBTY
title 2023, id 1nURL1_VBTpo48UX1tV24PXCvvgun3_7Eo

# 先搜索，后提示


In [17]:
from google.colab import drive, files
from google.colab import drive
from google.colab import auth
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google.oauth2.credentials import Credentials
import io
import json
import openai
from llama_index import Document, GPTSimpleVectorIndex
import os

auth.authenticate_user()

# Build the Drive API client
drive_service = build('drive', 'v3')
file_id = '14x1k7ErttIPXRcKN53g_Ogn3zfn6AEM_'
file = drive_service.files().get(fileId=file_id).execute()
content = drive_service.files().get_media(fileId=file_id).execute()

# Read the env config.json. set env for OPENAI_API_KEY
with io.BytesIO(content) as f:
    config = json.load(f)
os.environ["OPENAI_API_KEY"] = config["api_key"];
openai.api_key = os.environ["OPENAI_API_KEY"] ;

## 藤野先生txt文档
data_file_id = "1Z_X79_FrTW69h4SND9BnT9qr3ynsdl5G"
data_file = drive_service.files().get(fileId=data_file_id).execute()
content = drive_service.files().get_media(fileId=data_file_id).execute().decode()
# print(content)

doc = Document(content)

# print(doc)
# # create an index from the document object
index = GPTSimpleVectorIndex.from_documents([doc])


# # save the index to disk
response = index.query("鲁迅先生在日本学习医学的老师是谁？")
print(response)





鲁迅先生在日本学习医学的老师是藤野严九郎，他曾经问过鲁迅关于中国女性裹脚的情况，并且有一次派学生会干事到鲁迅寓里来借讲义看，之后邮差送来一封厚信，第一句是："——"。
